In [4]:
import os
import pandas as pd
import numpy as np  # make sure to add
import requests
from bs4 import BeautifulSoup
import logging  # make sure to add
import calendar  # make sure to add
from headers import headers  # change to your own headers file or add variable in code
from edgar_functions import *
import json
#tickers = ['AAPL', 'GOOGL', 'MSFT', 'AMZN'] 

In [89]:

data = pd.read_csv('tickers.csv', header=None)
tickers = data.values.T[0].tolist()
#ticker = ticker[60:64]
tickers

['AGX',
 'CRUS',
 'LEA',
 'MXL',
 'FN',
 'KLAC',
 'SWKS',
 'ON',
 'AOSL',
 'RMBS',
 'MRVL',
 'MCHP',
 'TXN',
 'SMTC',
 'SLAB',
 'ASML',
 'MASI',
 'AMD',
 'MPWR',
 'MU',
 'WDC',
 'LRCX',
 'STX',
 'INTC',
 'NXPI',
 'QCOM',
 'DORM',
 'AMCX']

In [100]:
tickers=['WDC',
 'LRCX',
 'STX',
 'INTC',
 'NXPI',
 'QCOM',

 'AMCX']






TWOU
PSTG
NTNX
BOX
APPF
WPP
ACM
KBR
MTZ
BBU
JEC
GVA
FLR
TUP
RICK
TAP
PEP
FLO
BUD
CLX
KO
CHD





## Second try 

In [127]:
def get_statements_for_tickers(tickers, statement_name):
    """
    Retrieves financial statements for multiple tickers.

    Args:
        tickers (list): List of stock ticker symbols.
        statement_name (str): Name of the financial statement.

    Returns:
        pd.DataFrame: Concatenated DataFrame containing financial statements for all tickers.
    """
    dfs = []  # List to store DataFrames for each ticker
    for ticker in tickers:
        try:
            # Get accession number
            accn = get_filtered_filings(ticker, ten_k=False, just_accession_numbers=True, headers=headers)
            acc_num = accn.iloc[0].replace('-', '')

            # Process statement
            df = process_one_statement(ticker, acc_num, statement_name)
            
            # Append DataFrame to list with ticker column
            if df is not None:
                df['Ticker'] = ticker  # Add ticker column
                dfs.append(df)
            else:
                print(f"Failed to process statement for ticker: {ticker}")
        except Exception as e:
            print(f"Error processing ticker {ticker}: {e}")

    # Concatenate DataFrames for all tickers
    if len(dfs) > 0:
        return pd.concat(dfs)
    else:
        return None

# Example usage:
tickers = [ 'HA']  # Example list of tickers
statement_name ='balance_sheet'  #"income_statement" #'balance_sheet'  # Example statement name
result_df = get_statements_for_tickers(tickers, statement_name)
result_df


,2024-03-31 00:00:00,2023-03-31 00:00:00,Ticker
us-gaap_StatementOfCashFlowsAbstract,NaN,NaN,HA
us-gaap_NetCashProvidedByUsedInOperatingActivities,254,"118,291",HA
us-gaap_PaymentsToAcquirePropertyPlantAndEquipment,"-127,018","-106,215",HA
us-gaap_ProceedsFromSaleOfPropertyPlantAndEquipment,105,"9,563",HA
us-gaap_PaymentsToAcquireInvestments,"-15,824","-96,806",HA
us-gaap_ProceedsFromSaleMaturityAndCollectionsOfInvestments,"109,485","144,069",HA
us-gaap_NetCashProvidedByUsedInInvestingActivities,"-33,252","-49,389",HA
us-gaap_ProceedsFromIssuanceOfLongTermDebt,"131,400",0,HA
us-gaap_RepaymentsOfDebtAndCapitalLeaseObligations,"-18,760","-24,953",HA
us-gaap_PaymentsOfDebtIssuanceCosts,"-1,849",0,HA


In [124]:
result_df = result_df.reset_index()
result_df[['taxonomy', 'fact_taxonomy']] = result_df['index'].str.split('_', expand=True)


In [125]:
import re
def split_camel_case(s):
    return ' '.join(re.findall(r'[A-Z][a-z]*', s))

# Apply function to 'fact_taxonomy' column
result_df['fact_taxonomy'] = result_df['fact_taxonomy'].apply(split_camel_case)
result_df = result_df.drop('index', axis = 1)

In [126]:
# Reverse the order of columns
reversed_columns = list(reversed(result_df.columns))

# Reorder columns using reindex
result_df = result_df.reindex(columns=reversed_columns)

# Print or use the DataFrame with reversed columns

result_df

,fact_taxonomy,taxonomy,Ticker,2023-03-31 00:00:00,2024-03-31 00:00:00
0,Revenues Abstract,us-gaap,HA,NaN,NaN
1,Revenue From Contract With Customer Excluding ...,us-gaap,HA,"612,603","645,567"
2,Labor And Related Expense,us-gaap,HA,"241,933","261,935"
3,Fuel Costs,us-gaap,HA,"197,625","188,778"
4,Aircraft Maintenance Materials And Repairs,us-gaap,HA,"50,287","70,971"
5,Costs Of Goods And Services Sold Aircraft And ...,ha,HA,"42,532","45,424"
6,Depreciation Depletion And Amortization,us-gaap,HA,"32,667","32,967"
7,Aircraft Rental,us-gaap,HA,"28,171","29,706"
8,Selling Expense,us-gaap,HA,"28,238","28,443"
9,Landing Fees And Other Rentals,us-gaap,HA,"38,720","43,127"


In [122]:
result_df.to_csv('ZZ_B.csv')

In [ ]:
#####